In [253]:
import numpy as np
import nltk
import pickle
import pandas as pd
import sys
import keras

import os
# mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-7.1.0-posix-seh-rt_v5-rev0\\mingw64\\bin'
# os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
    
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [254]:
# train_df = pd.read_csv("btc_train.csv")
train_df = pd.read_csv("eth_huobi.csv")

In [255]:
y_raw = train_df["last_price"]
x_raw = train_df.values[:,1:]
x_raw = scaler.fit_transform(x_raw)

log_count = len(y_raw)
shift = 150
shift_y = 150
x_train = np.zeros((log_count-shift-shift_y,shift,16))
y_train = np.zeros(log_count-shift-shift_y)
y_train_trend = np.zeros(log_count-shift-shift_y)

/home/daniel/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [256]:
for i in range(log_count-shift_y-shift):
    x_train[i] = x_raw[i:i+shift]
    y_train[i] = y_raw[i+shift+shift_y]
    y_train_trend[i] = y_raw[i+shift] <= y_raw[i+shift+shift_y]

In [259]:
# model = Sequential()
# model.add(LSTM(32,input_shape=(shift,15), activation='relu', return_sequences=False))
# model.add(Dense(1, activation='linear'))
# model.compile(loss='mae', optimizer='adam', metrics=['mean_absolute_percentage_error'])
# print(model.summary())

In [260]:
x_train_xgb = x_train.reshape(-1, x_train.shape[1]*x_train.shape[2])
split = int(x_train_xgb.shape[0]*0.8)
x_test = x_train_xgb[split:]
y_test = y_train[split:]
y_test_trend = y_train_trend[split:]
x_train = x_train_xgb[:split]
y_train = y_train[:split]
y_train_trend = y_train_trend[:split]

In [233]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

params = {"objective": "reg:linear", "booster":"gbtree", 'max_depth':'2', 'eta':'0.0025', 'subsample':'0.7', 'eval_metric':'mae' , 'verbose':1}
params['nthread'] = 3  
evallist  = [(dtest,'eval')]
num_round = 1500
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

[0]	eval-mae:400.069
[1]	eval-mae:399.05
[2]	eval-mae:398.034
[3]	eval-mae:397.021
[4]	eval-mae:396.01
[5]	eval-mae:395.001
[6]	eval-mae:393.996
[7]	eval-mae:392.992
[8]	eval-mae:391.991
[9]	eval-mae:390.993
[10]	eval-mae:389.998
[11]	eval-mae:389.004
[12]	eval-mae:388.013
[13]	eval-mae:387.025
[14]	eval-mae:386.039
[15]	eval-mae:385.055
[16]	eval-mae:384.074
[17]	eval-mae:383.096
[18]	eval-mae:382.12
[19]	eval-mae:381.146
[20]	eval-mae:380.175
[21]	eval-mae:379.206
[22]	eval-mae:378.239
[23]	eval-mae:377.275
[24]	eval-mae:376.314
[25]	eval-mae:375.355
[26]	eval-mae:374.398
[27]	eval-mae:373.443
[28]	eval-mae:372.492
[29]	eval-mae:371.542
[30]	eval-mae:370.595
[31]	eval-mae:369.65
[32]	eval-mae:368.708
[33]	eval-mae:367.768
[34]	eval-mae:366.83
[35]	eval-mae:365.894
[36]	eval-mae:364.961
[37]	eval-mae:364.03
[38]	eval-mae:363.102
[39]	eval-mae:362.175
[40]	eval-mae:361.252
[41]	eval-mae:360.33
[42]	eval-mae:359.411
[43]	eval-mae:358.494
[44]	eval-mae:357.579
[45]	eval-mae:356.667
[46]	

KeyboardInterrupt: 

In [249]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
rf = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=0)
rf.fit(x_train, y_train)
# RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#            max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
print(sum(rf.feature_importances_>0.01))

11


In [193]:
a = x_train[:,rf.feature_importances_>0]
b = x_test[:,rf.feature_importances_>0]
dtrain = xgb.DMatrix(a, label=y_train)
dtest = xgb.DMatrix(b, label=y_test)

params = {"objective": "reg:linear", "booster":"gbtree", 'max_depth':'2', 'eta':'0.003', 'subsample':'0.7', 'eval_metric':'mae' , 'verbose':1}
params['nthread'] = 3  
evallist  = [(dtest,'eval')]
num_round = 1500
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

[0]	eval-mae:399.895
[1]	eval-mae:398.674
[2]	eval-mae:397.456
[3]	eval-mae:396.242
[4]	eval-mae:395.032
[5]	eval-mae:393.826
[6]	eval-mae:392.623
[7]	eval-mae:391.424
[8]	eval-mae:390.228
[9]	eval-mae:389.036
[10]	eval-mae:387.847
[11]	eval-mae:386.662
[12]	eval-mae:385.481
[13]	eval-mae:384.303
[14]	eval-mae:383.129
[15]	eval-mae:381.958
[16]	eval-mae:380.791
[17]	eval-mae:379.627
[18]	eval-mae:378.466
[19]	eval-mae:377.31
[20]	eval-mae:376.156
[21]	eval-mae:375.006
[22]	eval-mae:373.86
[23]	eval-mae:372.717
[24]	eval-mae:371.577
[25]	eval-mae:370.441
[26]	eval-mae:369.308
[27]	eval-mae:368.178
[28]	eval-mae:367.052
[29]	eval-mae:365.93
[30]	eval-mae:364.81
[31]	eval-mae:363.694
[32]	eval-mae:362.582
[33]	eval-mae:361.473
[34]	eval-mae:360.367
[35]	eval-mae:359.264
[36]	eval-mae:358.164
[37]	eval-mae:357.068
[38]	eval-mae:355.976
[39]	eval-mae:354.886
[40]	eval-mae:353.8
[41]	eval-mae:352.717
[42]	eval-mae:351.637
[43]	eval-mae:350.561
[44]	eval-mae:349.487
[45]	eval-mae:348.417
[46]

[363]	eval-mae:129.433
[364]	eval-mae:129.022
[365]	eval-mae:128.612
[366]	eval-mae:128.203
[367]	eval-mae:127.796
[368]	eval-mae:127.39
[369]	eval-mae:126.986
[370]	eval-mae:126.582
[371]	eval-mae:126.18
[372]	eval-mae:125.778
[373]	eval-mae:125.378
[374]	eval-mae:124.979
[375]	eval-mae:124.582
[376]	eval-mae:124.185
[377]	eval-mae:123.789
[378]	eval-mae:123.395
[379]	eval-mae:123.003
[380]	eval-mae:122.611
[381]	eval-mae:122.221
[382]	eval-mae:121.831
[383]	eval-mae:121.443
[384]	eval-mae:121.056
[385]	eval-mae:120.67
[386]	eval-mae:120.285
[387]	eval-mae:119.902
[388]	eval-mae:119.519
[389]	eval-mae:119.138
[390]	eval-mae:118.757
[391]	eval-mae:118.379
[392]	eval-mae:118.001
[393]	eval-mae:117.624
[394]	eval-mae:117.248
[395]	eval-mae:116.874
[396]	eval-mae:116.5
[397]	eval-mae:116.128
[398]	eval-mae:115.757
[399]	eval-mae:115.387
[400]	eval-mae:115.018
[401]	eval-mae:114.65
[402]	eval-mae:114.283
[403]	eval-mae:113.917
[404]	eval-mae:113.553
[405]	eval-mae:113.189
[406]	eval-mae:11

[721]	eval-mae:39.0877
[722]	eval-mae:38.9478
[723]	eval-mae:38.808
[724]	eval-mae:38.6683
[725]	eval-mae:38.5492
[726]	eval-mae:38.4102
[727]	eval-mae:38.2717
[728]	eval-mae:38.1339
[729]	eval-mae:38.0166
[730]	eval-mae:37.8986
[731]	eval-mae:37.762
[732]	eval-mae:37.6256
[733]	eval-mae:37.5087
[734]	eval-mae:37.3919
[735]	eval-mae:37.2568
[736]	eval-mae:37.122
[737]	eval-mae:36.9873
[738]	eval-mae:36.8535
[739]	eval-mae:36.72
[740]	eval-mae:36.5863
[741]	eval-mae:36.4534
[742]	eval-mae:36.3208
[743]	eval-mae:36.1889
[744]	eval-mae:36.0764
[745]	eval-mae:35.9453
[746]	eval-mae:35.8335
[747]	eval-mae:35.7035
[748]	eval-mae:35.5914
[749]	eval-mae:35.4618
[750]	eval-mae:35.3514
[751]	eval-mae:35.2415
[752]	eval-mae:35.1129
[753]	eval-mae:34.9851
[754]	eval-mae:34.8572
[755]	eval-mae:34.7302
[756]	eval-mae:34.6035
[757]	eval-mae:34.477
[758]	eval-mae:34.3512
[759]	eval-mae:34.2256
[760]	eval-mae:34.1005
[761]	eval-mae:33.9755
[762]	eval-mae:33.8691
[763]	eval-mae:33.7448
[764]	eval-mae:33

[1076]	eval-mae:9.2485
[1077]	eval-mae:9.19865
[1078]	eval-mae:9.14863
[1079]	eval-mae:9.1001
[1080]	eval-mae:9.05073
[1081]	eval-mae:9.00153
[1082]	eval-mae:8.95255
[1083]	eval-mae:8.90372
[1084]	eval-mae:8.86539
[1085]	eval-mae:8.81793
[1086]	eval-mae:8.7705
[1087]	eval-mae:8.72239
[1088]	eval-mae:8.6743
[1089]	eval-mae:8.62653
[1090]	eval-mae:8.57962
[1091]	eval-mae:8.53322
[1092]	eval-mae:8.4866
[1093]	eval-mae:8.43915
[1094]	eval-mae:8.39298
[1095]	eval-mae:8.34578
[1096]	eval-mae:8.29866
[1097]	eval-mae:8.25175
[1098]	eval-mae:8.20554
[1099]	eval-mae:8.16809
[1100]	eval-mae:8.12211
[1101]	eval-mae:8.07567
[1102]	eval-mae:8.03041
[1103]	eval-mae:7.98412
[1104]	eval-mae:7.9383
[1105]	eval-mae:7.89242
[1106]	eval-mae:7.84668
[1107]	eval-mae:7.81092
[1108]	eval-mae:7.76612
[1109]	eval-mae:7.72145
[1110]	eval-mae:7.67637
[1111]	eval-mae:7.63125
[1112]	eval-mae:7.5864
[1113]	eval-mae:7.5417
[1114]	eval-mae:7.4976
[1115]	eval-mae:7.45302
[1116]	eval-mae:7.40869
[1117]	eval-mae:7.36455
[

[1416]	eval-mae:1.1786
[1417]	eval-mae:1.18948
[1418]	eval-mae:1.20101
[1419]	eval-mae:1.21198
[1420]	eval-mae:1.22372
[1421]	eval-mae:1.23534
[1422]	eval-mae:1.2473
[1423]	eval-mae:1.25974
[1424]	eval-mae:1.26991
[1425]	eval-mae:1.28221
[1426]	eval-mae:1.29439
[1427]	eval-mae:1.30698
[1428]	eval-mae:1.31707
[1429]	eval-mae:1.32838
[1430]	eval-mae:1.33898
[1431]	eval-mae:1.35038
[1432]	eval-mae:1.36183
[1433]	eval-mae:1.37339
[1434]	eval-mae:1.38383
[1435]	eval-mae:1.39621
[1436]	eval-mae:1.40864
[1437]	eval-mae:1.41941
[1438]	eval-mae:1.43097
[1439]	eval-mae:1.44166
[1440]	eval-mae:1.45422
[1441]	eval-mae:1.46706
[1442]	eval-mae:1.47877
[1443]	eval-mae:1.4915
[1444]	eval-mae:1.50355
[1445]	eval-mae:1.51623
[1446]	eval-mae:1.529
[1447]	eval-mae:1.5416
[1448]	eval-mae:1.55237
[1449]	eval-mae:1.5645
[1450]	eval-mae:1.57523
[1451]	eval-mae:1.58838
[1452]	eval-mae:1.5993
[1453]	eval-mae:1.60997
[1454]	eval-mae:1.62051
[1455]	eval-mae:1.63323
[1456]	eval-mae:1.64507
[1457]	eval-mae:1.65749


In [261]:
mean_absolute_error((gbm_1.predict(dtest))/1, y_test)

401.1474170201652

In [252]:
dtrain = xgb.DMatrix(x_train[:,rf.feature_importances_>0], label=y_train_trend)
dtest = xgb.DMatrix(x_test[:,rf.feature_importances_>0], label=y_test_trend)

params = {"objective": "binary:logistic", "booster":"gbtree", 'max_depth':'2', 'eta':'0.0025', 'subsample':'0.7', 'eval_metric':'error' , 'verbose':1}
params['nthread'] = 3  
evallist  = [(dtest,'eval')]
num_round = 1500
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

[0]	eval-error:0.781915
[1]	eval-error:0.781915
[2]	eval-error:0.787234
[3]	eval-error:0.781915
[4]	eval-error:0.787234
[5]	eval-error:0.781915
[6]	eval-error:0.781915
[7]	eval-error:0.781915
[8]	eval-error:0.781915
[9]	eval-error:0.781915
[10]	eval-error:0.781915
[11]	eval-error:0.781915
[12]	eval-error:0.781915
[13]	eval-error:0.781915
[14]	eval-error:0.781915
[15]	eval-error:0.781915
[16]	eval-error:0.781915
[17]	eval-error:0.781915
[18]	eval-error:0.781915
[19]	eval-error:0.787234
[20]	eval-error:0.781915
[21]	eval-error:0.781915
[22]	eval-error:0.781915
[23]	eval-error:0.781915
[24]	eval-error:0.787234
[25]	eval-error:0.781915
[26]	eval-error:0.781915
[27]	eval-error:0.781915
[28]	eval-error:0.781915
[29]	eval-error:0.787234
[30]	eval-error:0.781915
[31]	eval-error:0.781915
[32]	eval-error:0.781915
[33]	eval-error:0.781915
[34]	eval-error:0.781915
[35]	eval-error:0.781915
[36]	eval-error:0.781915
[37]	eval-error:0.781915
[38]	eval-error:0.781915
[39]	eval-error:0.781915
[40]	eval-

[320]	eval-error:0.781915
[321]	eval-error:0.781915
[322]	eval-error:0.781915
[323]	eval-error:0.781915
[324]	eval-error:0.781915
[325]	eval-error:0.781915
[326]	eval-error:0.781915
[327]	eval-error:0.781915
[328]	eval-error:0.781915
[329]	eval-error:0.781915
[330]	eval-error:0.781915
[331]	eval-error:0.781915
[332]	eval-error:0.781915
[333]	eval-error:0.781915
[334]	eval-error:0.781915
[335]	eval-error:0.781915
[336]	eval-error:0.781915
[337]	eval-error:0.781915
[338]	eval-error:0.781915
[339]	eval-error:0.781915
[340]	eval-error:0.781915
[341]	eval-error:0.781915
[342]	eval-error:0.781915
[343]	eval-error:0.781915
[344]	eval-error:0.781915
[345]	eval-error:0.781915
[346]	eval-error:0.781915
[347]	eval-error:0.781915
[348]	eval-error:0.781915
[349]	eval-error:0.781915
[350]	eval-error:0.781915
[351]	eval-error:0.781915
[352]	eval-error:0.781915
[353]	eval-error:0.781915
[354]	eval-error:0.781915
[355]	eval-error:0.781915
[356]	eval-error:0.781915
[357]	eval-error:0.781915
[358]	eval-e

[636]	eval-error:0.632979
[637]	eval-error:0.632979
[638]	eval-error:0.632979
[639]	eval-error:0.632979
[640]	eval-error:0.632979
[641]	eval-error:0.632979
[642]	eval-error:0.632979
[643]	eval-error:0.632979
[644]	eval-error:0.632979
[645]	eval-error:0.632979
[646]	eval-error:0.632979
[647]	eval-error:0.632979
[648]	eval-error:0.632979
[649]	eval-error:0.632979
[650]	eval-error:0.632979
[651]	eval-error:0.632979
[652]	eval-error:0.632979
[653]	eval-error:0.632979
[654]	eval-error:0.632979
[655]	eval-error:0.632979
[656]	eval-error:0.632979
[657]	eval-error:0.632979
[658]	eval-error:0.632979
[659]	eval-error:0.632979
[660]	eval-error:0.632979
[661]	eval-error:0.632979
[662]	eval-error:0.632979
[663]	eval-error:0.632979
[664]	eval-error:0.632979
[665]	eval-error:0.632979
[666]	eval-error:0.632979
[667]	eval-error:0.632979
[668]	eval-error:0.632979
[669]	eval-error:0.632979
[670]	eval-error:0.632979
[671]	eval-error:0.632979
[672]	eval-error:0.632979
[673]	eval-error:0.632979
[674]	eval-e

[959]	eval-error:0.62766
[960]	eval-error:0.62766
[961]	eval-error:0.62766
[962]	eval-error:0.62766
[963]	eval-error:0.62766
[964]	eval-error:0.62766
[965]	eval-error:0.62766
[966]	eval-error:0.62766
[967]	eval-error:0.62766
[968]	eval-error:0.62766
[969]	eval-error:0.62766
[970]	eval-error:0.62766
[971]	eval-error:0.62766
[972]	eval-error:0.62766
[973]	eval-error:0.62766
[974]	eval-error:0.62766
[975]	eval-error:0.62766
[976]	eval-error:0.62766
[977]	eval-error:0.62766
[978]	eval-error:0.62766
[979]	eval-error:0.62766
[980]	eval-error:0.62766
[981]	eval-error:0.62766
[982]	eval-error:0.62766
[983]	eval-error:0.62766
[984]	eval-error:0.62766
[985]	eval-error:0.62766
[986]	eval-error:0.62766
[987]	eval-error:0.62766
[988]	eval-error:0.62766
[989]	eval-error:0.62766
[990]	eval-error:0.62766
[991]	eval-error:0.62766
[992]	eval-error:0.62766
[993]	eval-error:0.62766
[994]	eval-error:0.62766
[995]	eval-error:0.62766
[996]	eval-error:0.62766
[997]	eval-error:0.62766
[998]	eval-error:0.62766


[1276]	eval-error:0.542553
[1277]	eval-error:0.542553
[1278]	eval-error:0.547872
[1279]	eval-error:0.547872
[1280]	eval-error:0.547872
[1281]	eval-error:0.542553
[1282]	eval-error:0.547872
[1283]	eval-error:0.547872
[1284]	eval-error:0.547872
[1285]	eval-error:0.547872
[1286]	eval-error:0.547872
[1287]	eval-error:0.542553
[1288]	eval-error:0.542553
[1289]	eval-error:0.547872
[1290]	eval-error:0.547872
[1291]	eval-error:0.547872
[1292]	eval-error:0.547872
[1293]	eval-error:0.547872
[1294]	eval-error:0.547872
[1295]	eval-error:0.547872
[1296]	eval-error:0.547872
[1297]	eval-error:0.547872
[1298]	eval-error:0.547872
[1299]	eval-error:0.547872
[1300]	eval-error:0.553191
[1301]	eval-error:0.547872
[1302]	eval-error:0.553191
[1303]	eval-error:0.547872
[1304]	eval-error:0.542553
[1305]	eval-error:0.547872
[1306]	eval-error:0.547872
[1307]	eval-error:0.542553
[1308]	eval-error:0.542553
[1309]	eval-error:0.547872
[1310]	eval-error:0.547872
[1311]	eval-error:0.547872
[1312]	eval-error:0.547872
[